<a href="https://colab.research.google.com/github/mounikamarreddy/BERTFinetuning-NLPTasks/blob/main/finetuneBERT_RTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we build a deep learning model to perform RTE (Recognizing Textual Entailment) task. RTE is classifying relationships between pairs of sentences as entailment or not. First, we will develop a preliminary model by fine-tuning a pretrained BERT.

In [1]:
!pip install transformers
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.9 MB/s 
     |████████████████████████████████| 120 kB 43.2 MB/s 
     |████████████████████████████████| 6.6 MB 42.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [4]:
!wget https://dl.fbaipublicfiles.com/glue/data/RTE.zip
!unzip RTE.zip
!ls

--2022-09-03 19:07:47--  https://dl.fbaipublicfiles.com/glue/data/RTE.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 697150 (681K) [application/zip]
Saving to: ‘RTE.zip’

RTE.zip             100%[===================>] 680.81K  --.-KB/s    in 0.03s   

2022-09-03 19:07:48 (22.5 MB/s) - ‘RTE.zip’ saved [697150/697150]

Archive:  RTE.zip
   creating: RTE/
  inflating: RTE/dev.tsv             
  inflating: RTE/test.tsv            
  inflating: RTE/train.tsv           
RTE  RTE.zip  sample_data


In [14]:
df = pd.read_csv('RTE/train.tsv',error_bad_lines=False, sep='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
df.head()

,index,sentence1,sentence2,label
0,0,No Weapons of Mass Destruction Found in Iraq Yet.,Weapons of Mass Destruction Found in Iraq.,not_entailment
1,1,"A place of sorrow, after Pope John Paul II die...",Pope Benedict XVI is the new leader of the Rom...,entailment
2,2,Herceptin was already approved to treat the si...,Herceptin can be used to treat breast cancer.,entailment
3,3,"Judie Vivian, chief executive at ProMedica, a ...",The previous name of Ho Chi Minh City was Saigon.,entailment
4,4,A man is due in court later charged with the m...,Paul Stewart Hutchinson is accused of having s...,not_entailment


In [16]:
df.label.value_counts()

entailment        1249
not_entailment    1240
Name: label, dtype: int64

In [17]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(df['label'])
df['label_numeric'] = encoder.transform(df['label'])

In [18]:
df.label_numeric.value_counts()

0    1249
1    1240
2       1
Name: label_numeric, dtype: int64

In [19]:
df = df[df.label_numeric!=2] #removing label where we have 1 value (2)
df.reset_index(drop=True, inplace=True)
print(df.shape)

(2489, 5)


In [20]:
# Create sentence and label lists
sentences1 = df['sentence1'].values
sentences2 = df['sentence2'].values

# We need to add special tokens at the beginning, between sentences and end of each sentence for BERT to work properly
sentences = ["[CLS] " + str(sentences1[i]) + " [SEP] " + str(sentences2[i]) + " [SEP]" for i in np.arange(df.shape[0])]
labels = df['label_numeric'].values

In [21]:
np.unique(labels)

array([0, 1])

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(sentences[0])
print ("Tokenize the both sentences:")
print (tokenized_texts[0])

[CLS] No Weapons of Mass Destruction Found in Iraq Yet. [SEP] Weapons of Mass Destruction Found in Iraq. [SEP]
Tokenize the both sentences:
['[CLS]', 'no', 'weapons', 'of', 'mass', 'destruction', 'found', 'in', 'iraq', 'yet', '.', '[SEP]', 'weapons', 'of', 'mass', 'destruction', 'found', 'in', 'iraq', '.', '[SEP]']


In [23]:
# Set the maximum sequence length. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

In [24]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
print(len(input_ids[0]))

21


In [25]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(len(input_ids[0]))

128


In [26]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [28]:
# create token_types or segmentation masks
encoded_dict = []
for i in np.arange(df.shape[0]):
  input_ids1 = tokenizer(sentences1[i], sentences2[i])
  input_ids1 = pad_sequences(np.array(input_ids1['token_type_ids']).reshape(1,len(input_ids1['token_type_ids'])), maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  encoded_dict.append(input_ids1)

In [29]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2020, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2020, test_size=0.1)
train_seg_masks, validation_seg_masks, _, _ = train_test_split(encoded_dict, input_ids,
                                             random_state=2020, test_size=0.1)

In [30]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
train_seg_masks = torch.tensor(np.array(train_seg_masks).reshape(np.array(train_seg_masks).shape[0],np.array(train_seg_masks).shape[2]))
validation_seg_masks = torch.tensor(np.array(validation_seg_masks).reshape(np.array(validation_seg_masks).shape[0],np.array(validation_seg_masks).shape[2]))

In [31]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 8

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_seg_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_seg_masks,  validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [32]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3, output_hidden_states=True)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [33]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [34]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [35]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [36]:
t = [] 

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (recommend between 2 and 4)
epochs = 1
loss=torch.nn.CrossEntropyLoss()
# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    #print(step, batch)
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # batch = tuple(t for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_input_seg_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids.long(), token_type_ids=b_input_seg_mask, attention_mask=b_input_mask, labels=b_labels)[0]

    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    #batch = tuple(t for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_input_seg_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids.long(), token_type_ids=b_input_seg_mask, attention_mask=b_input_mask)[0]
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Train loss: 0.733098789198058


Epoch: 100%|██████████| 1/1 [00:37<00:00, 37.58s/it]

Validation Accuracy: 0.62109375


In [40]:
df_test = pd.read_csv('RTE/dev.tsv',error_bad_lines=False, sep='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
df_test.head()

,index,sentence1,sentence2,label
0,0,"Dana Reeve, the widow of the actor Christopher...",Christopher Reeve had an accident.,not_entailment
1,1,"Yet, we now are discovering that antibiotics a...",Bacteria is winning the war against antibiotics.,entailment
2,2,Cairo is now home to some 15 million people - ...,15 million tonnes of rubbish are produced dail...,not_entailment
3,3,"The Amish community in Pennsylvania, which num...",Pennsylvania has the biggest Amish community i...,not_entailment
4,4,Security forces were on high alert after an el...,Security forces were on high alert after a cam...,entailment


In [47]:
df_test['label_numeric'] = encoder.transform(df_test['label'])

In [49]:
# Create sentence and label lists
sentences1 = df_test['sentence1'].values
sentences2 = df_test['sentence2'].values
# We need to add special tokens at the beginning, between sentences and end of each sentence for BERT to work properly
sentences = ["[CLS] " + str(sentences1[i]) + " [SEP] " + str(sentences2[i]) + " [SEP]" for i in np.arange(df_test.shape[0])]
labels = df_test['label_numeric'].values

In [50]:
np.unique(labels)

array([0, 1])

In [51]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(sentences[0])
print ("Tokenize the both sentences:")
print (tokenized_texts[0])

[CLS] Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation. [SEP] Christopher Reeve had an accident. [SEP]
Tokenize the both sentences:
['[CLS]', 'dana', 'reeve', ',', 'the', 'widow', 'of', 'the', 'actor', 'christopher', 'reeve', ',', 'has', 'died', 'of', 'lung', 'cancer', 'at', 'age', '44', ',', 'according', 'to', 'the', 'christopher', 'reeve', 'foundation', '.', '[SEP]', 'christopher', 'reeve', 'had', 'an', 'accident', '.', '[SEP]']


In [52]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
print(len(input_ids[0]))

36


In [53]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(len(input_ids[0]))

128


In [54]:
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)  

In [55]:
encoded_dict = []
for i in np.arange(df_test.shape[0]):
  input_ids1 = tokenizer(sentences1[i], sentences2[i])
  input_ids1 = pad_sequences(np.array(input_ids1['token_type_ids']).reshape(1,len(input_ids1['token_type_ids'])), maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  encoded_dict.append(input_ids1)

In [57]:
test_ids = torch.tensor(input_ids)
test_masks = torch.tensor(attention_masks)
test_labels = torch.tensor(df_test['label_numeric'].values)
test_seg_masks = torch.tensor(np.array(encoded_dict).reshape(len(encoded_dict),128))

In [58]:
test_data = TensorDataset(test_ids, test_masks, test_seg_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [59]:
# Put model in evaluation mode to evaluate loss on the validation set
model.eval()
# Tracking variables
test_loss, test_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
# Evaluate data for one epoch
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  #batch = tuple(t for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_input_seg_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids.long(), token_type_ids=b_input_seg_mask, attention_mask=b_input_mask)[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  tmp_eval_accuracy = flat_accuracy(logits, label_ids)
  test_accuracy += tmp_eval_accuracy
  nb_eval_steps += 1
print("Test Accuracy: {}".format(test_accuracy/nb_eval_steps))

Test Accuracy: 0.6071428571428571
